# Preprocessing and split

In this notebook ECG and PCG signals are read. Preprocessing and feature extraction is performed. Train-test split is also performed, objects are exported to be loaded by the other scripts

#### Importing libraries

In [50]:
import wfdb
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from python_speech_features.base import mfcc
import scipy.io.wavfile as wav
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import GRU
import numpy as np
from scipy.ndimage import zoom
import pandas as pd
from gen_wavelet_images import generate_wavelet_images as gen_wav
import scipy

In [51]:
from sklearn import preprocessing

In [52]:
import pywt

In [53]:
from scipy.ndimage import zoom

In [54]:
import matplotlib
matplotlib.use('Agg')

In [55]:
from scipy import signal

In [56]:
import concurrent.futures
import multiprocessing as mp
mp.set_start_method("spawn")


RuntimeError: context has already been set

#### Constants definition

In [57]:
RESAMPLE_FACTOR = 1 #The downsampling factor to apply(1 means no downsampling is applied, 2 means the frequency is reduced to half)
TRAIN_SIZE = 0.7 # The training portion to apply in the train-test split
FRAME_TIME = 5 # The frame time (in seconds) to divide the signals into equally sized samples
FRAME_OVERLAP = 0 # The frame overlap (in seconds)

#### Reading the signals

In [58]:
cont = 0
length_array = []
multimodal_records = []
print ('Only 1 signal  in records: ')
for filename in os.listdir("training-a"):
    if filename.endswith(".hea") : 
        
        record = wfdb.rdheader('training-a/'+filename[:-4])
        
        if record.__dict__['n_sig'] == 1:
             print (filename)
           # print (record.__dict__)
        elif record.__dict__['n_sig'] == 2:
            cont +=1
            length = record.__dict__['sig_len']/record.__dict__['fs']
            length_array.append(length)
            #print ( record.__dict__['comments'])
            if record.__dict__['comments'] == ["Abnormal"]:
                multimodal_records.append((filename[:-4], 1))
            else:
                multimodal_records.append((filename[:-4], 0))
    else:
        continue
print (record.__dict__)
fs = record.__dict__['fs']
print ("Sampling frequency is: " + str(fs) + "Hz")

Only 1 signal  in records: 
a0041.hea
a0117.hea
a0220.hea
a0233.hea
{'record_name': 'a0409', 'n_sig': 2, 'fs': 2000, 'counter_freq': None, 'base_counter': None, 'sig_len': 61440, 'base_time': None, 'base_date': None, 'comments': ['Normal'], 'sig_name': ['PCG', 'ECG'], 'p_signal': None, 'd_signal': None, 'e_p_signal': None, 'e_d_signal': None, 'file_name': ['a0409.wav', 'a0409.dat'], 'fmt': ['16', '16'], 'samps_per_frame': [1, 1], 'skew': [None, None], 'byte_offset': [44, None], 'adc_gain': [1.0, 1000.0], 'baseline': [0, 0], 'units': ['mV', 'mV'], 'adc_res': [16, 16], 'adc_zero': [0, 0], 'init_value': [0, -3], 'checksum': [0, -28647], 'block_size': [0, 0]}
Sampling frequency is: 2000Hz


Signals will be resampled to 1000 Hz (?)

Out of the 409 signals records in dataset A, 405 are multimodal
Here the record identifier and the classification is saved
1 represents abnormal signals
0 represents normal signals

Both ECG and PCG are initially sampled at 2000 Hz (fs)

### Preprocessing

Signals can be down sampled by adjusting the resample factor

 Dropping the first 50 samples for both signals because in ECG the signal usually starts at 0 and in some cases jumps to a high baseline causing a huge transient that leads to problems on the wavelet boundary

<div>
<img src="y.png" width="500"/>
</div>

In [59]:
ecg = []
pcg = []
resample_factor = RESAMPLE_FACTOR
y = []
file_list = []
for file, target in multimodal_records:
    ecg_sig = wfdb.rdrecord('training-a/'+ file).__dict__['p_signal'][:,1]
    ecg.append(ecg_sig[50::resample_factor])
    y.append (target)
    rate,sig = wav.read("training-a\\"+ file+".wav")
    pcg.append(sig[50::resample_factor])
    file_list.append(file)
y = np.array(y)

Record wise: 70-30 train test split


In [60]:
X_train_ecg,  X_test_ecg, y_train, y_test, ind_train, ind_test = train_test_split (ecg, y, file_list, train_size = TRAIN_SIZE, random_state = 1, stratify = y)
X_train_pcg,  X_test_pcg, y_train, y_test, ind_train_pcg, ind_test_pcg = train_test_split (pcg, y, file_list, train_size = TRAIN_SIZE, random_state = 1, stratify=y)

In [61]:
print ('Train size', len(X_train_ecg))
print ('Test size', len(X_test_pcg))


Train size 283
Test size 122


In [62]:
fr = fs/resample_factor # desired frequency

Frame Wise separation with _frame_time_ second windows with padding and _frame_overal_ seconds overlap, train test split is done before so that frames from the same record end up in the same group

In [63]:
frame_time = FRAME_TIME # time in seconds desired
frame_overlap = FRAME_OVERLAP # overlap in seconds

frame_length = int (fr * frame_time)
frame_step = int(frame_length - fr * frame_overlap)


In [64]:
def create_frames (ecg, pcg, y):
    X_ecg = []
    f_y_train = []
    X_pcg = []
    record_id = []
    for index in range (len (ecg)):
        ecg_frames = tf.signal.frame(ecg[index], frame_length= frame_length, frame_step = frame_step, pad_end=False).numpy().tolist()
        X_ecg.extend (ecg_frames)
        X_pcg.extend(tf.signal.frame(pcg[index], frame_length= frame_length, frame_step = frame_step, pad_end=False).numpy().tolist())
        
        f_y_train = f_y_train + [y[index]] * len (ecg_frames)
        record_id.extend([index]* len (ecg_frames))
    return X_ecg, X_pcg, f_y_train, record_id


In [65]:
f_X_train_ecg,f_X_train_pcg, f_y_train, record_id = create_frames (X_train_ecg, X_train_pcg, y_train)

In [66]:
f_X_train_ecg= np.array(f_X_train_ecg)
f_X_train_pcg= np.array(f_X_train_pcg)


In [67]:
f_X_test_ecg, f_X_test_pcg, f_y_test, record_id_test = create_frames (X_test_ecg, X_test_pcg, y_test)

In [68]:
f_y_test, f_y_train = np.array(f_y_test), np.array(f_y_train)

In [69]:
f_X_test_ecg= np.array(f_X_test_ecg)
f_X_test_pcg= np.array(f_X_test_pcg)


# Normalization

In [21]:
plt.plot (f_X_train_ecg[0])

In [22]:
len (f_X_test_ecg)

769

In [23]:
def find_min (array):
    return min(elem for subls in array for elem in subls)
def find_max (array):
     return max(elem for subls in array for elem in subls)
def normalize (maximum, minimum, array):
    new_array = []
    for line in array:
        new_line = []
        for value in line:
            
            value = (value -minimum)/(maximum-minimum)
            new_line.append(value)
        new_array.append(new_line)
    return np.array (new_array)

In [24]:
def get_average (array):
    return np.mean([elem for subls in array for elem in subls])
def get_stdev (array):
     return np.std([elem for subls in array for elem in subls])
def standardize (avg, std, array):
    new_array = []
    for line in array:
        new_line = []
        for value in line:
            
            value = (value -avg)/std
            new_line.append(value)
        new_array.append(new_line)
    return np.array (new_array)

In [25]:
f_X_train_ecg = np.nan_to_num(f_X_train_ecg, copy=True, nan=0.0, posinf=None, neginf=None)
f_X_test_ecg  = np.nan_to_num(f_X_test_ecg, copy=True, nan=0.0, posinf=None, neginf=None)

In [26]:
def standardize_signals(signals):
    standardized_signals = []
    for signal in signals:
        mean = np.mean(signal)
        #print (mean)
        std = np.std(signal)
        standardized_signal = (signal - mean) / std
        standardized_signals.append(standardized_signal)
    return np.array(standardized_signals)

In [27]:

def butter_bandpass(lowcut, highcut, fs, order, signals):
    # Normalized cutoff frequencies
    b, a = scipy.signal.butter(order, [lowcut, highcut], btype='band', fs=fs)
    
    filtered_signals = []
    for s in signals:
        s= np.ravel(s)
        #print (s.shape)
        y = scipy.signal.filtfilt(b, a, s)
        
        filtered_signals.append(y)
    return np.array(filtered_signals)


In [28]:
def remove_spikes(signals, window_size=0.5, fs=2000):
    # Convert window size from seconds to samples
    window_size = int(window_size * fs)
    filtered_signals = []
    i = 0
    for signal in signals:
        i+=1
        print (i)
        # Initialize the cleaned signal
        signal= np.ravel(signal)
        signal_clean = signal.copy()
        # Compute the maximum absolute amplitude (MAA) for each sliding window
        while True:
            
            windows = [signal_clean[i:i+window_size] for i in range(0, len(signal_clean), window_size)]

            # Compute the maximum absolute amplitude (MAA) for each window
            maa = [np.max(np.abs(window)) for window in windows]
            
        
            # Compute the median MAA value
            median_maa = np.median(maa)
                    
            
            # Check if any MAA exceeds three times the median value of the MAA's

            if np.any(maa > 3 * median_maa):
                print ('entered')
                # Find the window with the highest MAA value
                max_maa_index  = np.argmax(maa)
                
                print (max_maa_index)
                spike_index = max_maa_index * window_size + np.argmax(np.abs(windows[max_maa_index]))
                
                print (spike_index)
                
                
                # Determine the first zero-crossing point after the spike index
                before_spike_index = spike_index -2 if spike_index>2 else 0
                after_spike_index  = spike_index +2 if len(signal)-spike_index>2 else len(signal)
                
                # Determine the last zero-crossing point before the spike index
               
                zero_crossings_before = np.where(np.diff(np.sign(signal[:before_spike_index])))[0]
                if len (zero_crossings_before)>0:
                    last_zero_crossing = zero_crossings_before[-1]
                else:
                    last_zero_crossing = 0
                    
                
                
                
                zero_crossings_after = np.where(np.diff(np.sign(signal[before_spike_index:])))[0]
                if len (zero_crossings_after)>0:
                    first_zero_crossing_after = zero_crossings_after[0] + spike_index
                else:
                    first_zero_crossing_after = len(signal_clean) - 1 
                # Replace the noise spike with zeros
                signal_clean[last_zero_crossing:first_zero_crossing_after +1] = 0
                print (last_zero_crossing,first_zero_crossing_after)
            else:
                print ('not entered')
                break
        filtered_signals.append(signal_clean)
    return np.array(filtered_signals)

In [29]:
f_X_train_pcg = standardize_signals(f_X_train_pcg)
f_X_test_pcg = standardize_signals(f_X_test_pcg)
f_X_train_ecg = standardize_signals(f_X_train_ecg)
f_X_test_ecg = standardize_signals(f_X_test_ecg)


In [153]:
plt.plot(x)
plt.savefig('ECG_noise.png')
plt.close()

In [152]:
x = f_X_test_ecg[32]

In [148]:
x2 = butter_bandpass(1, 100, 2000, 4, [x])

In [149]:
len(x2)

1

In [150]:
plt.plot(x2[0])
plt.savefig('ECG_denoise.png')
plt.close()

In [31]:


f_X_train_pcg = butter_bandpass(25, 400, 2000, 4, f_X_train_pcg)
f_X_test_pcg = butter_bandpass(25, 400, 2000, 4, f_X_test_pcg)

f_X_train_pcg = remove_spikes( f_X_train_pcg)
f_X_test_pcg = remove_spikes( f_X_test_pcg)


f_X_train_ecg = butter_bandpass(0.5, 100, 2000, 4, f_X_train_ecg)
f_X_test_ecg = butter_bandpass(0.5, 100, 2000, 4, f_X_test_ecg)

f_X_train_ecg = remove_spikes( f_X_train_ecg)
f_X_test_ecg = remove_spikes( f_X_test_ecg)

1
not entered
2
not entered
3
not entered
4
not entered
5
not entered
6
not entered
7
not entered
8
not entered
9
not entered
10
not entered
11
not entered
12
not entered
13
not entered
14
not entered
15
not entered
16
not entered
17
not entered
18
not entered
19
not entered
20
not entered
21
not entered
22
not entered
23
not entered
24
not entered
25
not entered
26
not entered
27
not entered
28
not entered
29
not entered
30
not entered
31
not entered
32
not entered
33
not entered
34
not entered
35
not entered
36
not entered
37
not entered
38
not entered
39
not entered
40
not entered
41
not entered
42
not entered
43
entered
8
8628
8620 8641
entered
6
6446
6437 6458
entered
2
2249
2238 2261
entered
4
4352
4339 4363
entered
4
4372
4361 4385
entered
2
2229
2217 2240
entered
8
8614
8590 8622
entered
4
4302
4292 4312
entered
6
6431
6420 6439
not entered
44
not entered
45
not entered
46
not entered
47
not entered
48
not entered
49
not entered
50
not entered
51
not entered
52
not entered
53
n

not entered
663
not entered
664
not entered
665
not entered
666
not entered
667
not entered
668
not entered
669
not entered
670
not entered
671
not entered
672
not entered
673
not entered
674
not entered
675
not entered
676
not entered
677
not entered
678
not entered
679
not entered
680
not entered
681
not entered
682
not entered
683
not entered
684
not entered
685
not entered
686
not entered
687
not entered
688
not entered
689
not entered
690
not entered
691
not entered
692
not entered
693
not entered
694
not entered
695
not entered
696
not entered
697
not entered
698
not entered
699
not entered
700
not entered
701
not entered
702
not entered
703
not entered
704
not entered
705
not entered
706
not entered
707
not entered
708
not entered
709
not entered
710
not entered
711
not entered
712
not entered
713
not entered
714
not entered
715
not entered
716
not entered
717
not entered
718
not entered
719
not entered
720
not entered
721
not entered
722
not entered
723
not entered
724
not ente

1180
not entered
1181
not entered
1182
not entered
1183
not entered
1184
entered
5
5816
5803 5827
not entered
1185
entered
1
1653
1648 1670
not entered
1186
not entered
1187
not entered
1188
not entered
1189
not entered
1190
not entered
1191
not entered
1192
not entered
1193
not entered
1194
not entered
1195
not entered
1196
not entered
1197
not entered
1198
entered
9
9082
9078 9089
not entered
1199
not entered
1200
not entered
1201
entered
9
9181
9162 9186
entered
1
1399
1370 1403
not entered
1202
not entered
1203
not entered
1204
not entered
1205
not entered
1206
not entered
1207
not entered
1208
not entered
1209
not entered
1210
not entered
1211
not entered
1212
not entered
1213
not entered
1214
not entered
1215
not entered
1216
not entered
1217
not entered
1218
not entered
1219
not entered
1220
not entered
1221
not entered
1222
not entered
1223
entered
5
5266
5261 5275
not entered
1224
not entered
1225
not entered
1226
not entered
1227
not entered
1228
not entered
1229
not entered


not entered
1780
not entered
1781
not entered
1782
not entered
1783
not entered
1784
not entered
1785
entered
8
8558
8553 8567
not entered
1786
not entered
1787
not entered
1788
not entered
1789
not entered
1790
not entered
1791
not entered
1792
entered
5
5613
5599 5623
entered
5
5629
5621 5648
not entered
1793
entered
1
1929
1918 1943
not entered
1
not entered
2
not entered
3
not entered
4
not entered
5
not entered
6
not entered
7
not entered
8
not entered
9
not entered
10
not entered
11
not entered
12
not entered
13
entered
0
2
0 2
entered
0
3
0 29
entered
4
4894
4883 4907
entered
4
4874
4862 4885
entered
7
7575
7565 7587
not entered
14
not entered
15
entered
2
2503
2493 2507
entered
2
2509
2505 2521
entered
2
2555
2551 2559
not entered
16
not entered
17
entered
1
1318
1306 1329
entered
6
6643
6629 6654
entered
1
1336
1327 1350
entered
9
9277
9266 9289
entered
6
6662
6652 6676
entered
3
3932
3922 3945
entered
9
9258
9244 9268
not entered
18
not entered
19
entered
5
5220
5207 5232
ent

not entered
427
not entered
428
not entered
429
not entered
430
not entered
431
not entered
432
not entered
433
not entered
434
not entered
435
not entered
436
not entered
437
not entered
438
not entered
439
not entered
440
not entered
441
not entered
442
not entered
443
not entered
444
not entered
445
not entered
446
not entered
447
not entered
448
not entered
449
not entered
450
not entered
451
not entered
452
not entered
453
not entered
454
not entered
455
not entered
456
not entered
457
not entered
458
not entered
459
not entered
460
not entered
461
not entered
462
not entered
463
not entered
464
not entered
465
entered
4
4068
4059 4080
entered
1
1839
1829 1851
entered
4
4090
4078 4102
not entered
466
not entered
467
entered
3
3863
3854 3876
entered
6
6147
6137 6160
entered
6
6127
6118 6139
entered
1
1616
1607 1626
entered
1
1636
1624 1647
entered
3
3886
3874 3899
entered
8
8322
8311 8332
entered
8
8304
8295 8313
not entered
468
not entered
469
not entered
470
not entered
471
not e

not entered
332
not entered
333
not entered
334
not entered
335
not entered
336
not entered
337
not entered
338
not entered
339
not entered
340
not entered
341
not entered
342
not entered
343
not entered
344
not entered
345
not entered
346
not entered
347
not entered
348
not entered
349
not entered
350
not entered
351
not entered
352
not entered
353
not entered
354
not entered
355
entered
1
1846
1596 2013
not entered
356
not entered
357
not entered
358
not entered
359
not entered
360
not entered
361
not entered
362
not entered
363
not entered
364
not entered
365
not entered
366
not entered
367
not entered
368
not entered
369
not entered
370
not entered
371
not entered
372
not entered
373
not entered
374
entered
7
7995
7962 8802
entered
7
7902
6588 7964
not entered
375
not entered
376
not entered
377
not entered
378
not entered
379
not entered
380
not entered
381
not entered
382
not entered
383
not entered
384
not entered
385
not entered
386
not entered
387
not entered
388
not entered
3

not entered
1011
not entered
1012
not entered
1013
not entered
1014
not entered
1015
not entered
1016
not entered
1017
not entered
1018
not entered
1019
not entered
1020
not entered
1021
not entered
1022
not entered
1023
not entered
1024
not entered
1025
not entered
1026
not entered
1027
not entered
1028
not entered
1029
entered
9
9906
9513 9999
not entered
1030
entered
0
453
66 1058
not entered
1031
not entered
1032
not entered
1033
not entered
1034
not entered
1035
not entered
1036
not entered
1037
not entered
1038
not entered
1039
not entered
1040
not entered
1041
not entered
1042
not entered
1043
not entered
1044
not entered
1045
not entered
1046
not entered
1047
not entered
1048
not entered
1049
not entered
1050
not entered
1051
not entered
1052
not entered
1053
not entered
1054
not entered
1055
not entered
1056
not entered
1057
not entered
1058
not entered
1059
not entered
1060
not entered
1061
not entered
1062
not entered
1063
not entered
1064
not entered
1065
not entered
1066
n

1
not entered
2
not entered
3
not entered
4
not entered
5
not entered
6
not entered
7
not entered
8
not entered
9
not entered
10
not entered
11
not entered
12
not entered
13
entered
9
9943
9603 9999
entered
7
7424
5981 7464
not entered
14
not entered
15
entered
8
8438
8390 8471
entered
0
514
457 545
entered
3
3116
3061 3152
entered
5
5741
5691 5774
entered
8
8498
8469 8736
entered
5
5808
5772 6068
entered
0
579
543 677
entered
3
3182
3150 3252
not entered
16
not entered
17
entered
9
9123
9072 9156
entered
6
6507
6459 6541
entered
3
3783
3732 3817
entered
1
1182
1130 1215
entered
6
6571
6539 6820
entered
1
1245
1213 1610
entered
9
9187
9154 9310
entered
3
3846
3815 4191
not entered
18
entered
9
9759
9709 9793
entered
7
7144
7084 7178
entered
1
1821
1772 1855
entered
4
4434
4372 4469
entered
9
9822
9791 9880
entered
4
4496
4467 4801
entered
7
7208
7176 7302
entered
1
1886
1853 1994
not entered
19
entered
5
5080
5024 5115
entered
2
2440
2396 2472
entered
7
7645
7601 7677
entered
2
2504
24

not entered
699
not entered
700
not entered
701
not entered
702
not entered
703
not entered
704
not entered
705
not entered
706
not entered
707
not entered
708
not entered
709
not entered
710
not entered
711
not entered
712
not entered
713
not entered
714
not entered
715
not entered
716
not entered
717
not entered
718
not entered
719
not entered
720
not entered
721
not entered
722
not entered
723
not entered
724
not entered
725
not entered
726
not entered
727
not entered
728
not entered
729
not entered
730
not entered
731
not entered
732
not entered
733
not entered
734
not entered
735
not entered
736
not entered
737
not entered
738
not entered
739
not entered
740
not entered
741
not entered
742
not entered
743
not entered
744
not entered
745
not entered
746
not entered
747
not entered
748
not entered
749
not entered
750
not entered
751
not entered
752
not entered
753
not entered
754
not entered
755
not entered
756
not entered
757
not entered
758
not entered
759
not entered
760
not ente

In [32]:
len (f_X_test_ecg)

769

### Feature extraction


In [33]:
f_mfcc_train = []
for sig in f_X_train_pcg:
    f_mfcc_train.append(np.array(mfcc (sig, fr, numcep=13, appendEnergy=False)))
f_mfcc_train = np.array(f_mfcc_train)
f_mfcc_train.shape
f_mfcc_train = f_mfcc_train.reshape(f_mfcc_train.shape[0], f_mfcc_train.shape[2], f_mfcc_train.shape[1], 1)

In [34]:
f_mfcc_test = []
for sig in f_X_test_pcg:
    f_mfcc_test.append(np.array(mfcc (sig, fr, numcep = 13, appendEnergy=False)))
f_mfcc_test = np.array(f_mfcc_test)
f_mfcc_test.shape
f_mfcc_test = f_mfcc_test.reshape(f_mfcc_test.shape[0], f_mfcc_test.shape[2], f_mfcc_test.shape[1], 1)

In [35]:
f_X_test_ecg= np.array(f_X_test_ecg)
f_X_test_pcg= np.array(f_X_test_pcg)

### Reshaping

In [36]:
f_X_train_ecg = np.nan_to_num(f_X_train_ecg, copy=True, nan=0.0, posinf=None, neginf=None)
f_X_test_ecg  = np.nan_to_num(f_X_test_ecg, copy=True, nan=0.0, posinf=None, neginf=None)

In [37]:
f_X_train_ecg = f_X_train_ecg.reshape(f_X_train_ecg.shape[0], f_X_train_ecg.shape[1], 1)
f_X_train_pcg = f_X_train_pcg.reshape(f_X_train_pcg.shape[0], f_X_train_pcg.shape[1], 1)
f_X_test_ecg = f_X_test_ecg.reshape(f_X_test_ecg.shape[0], f_X_test_ecg.shape[1], 1)
f_X_test_pcg = f_X_test_pcg.reshape(f_X_test_pcg.shape[0], f_X_test_pcg.shape[1], 1)

In [38]:
with open('f_X_test_pcg.npy', 'wb') as f:
    np.save(f, f_X_test_pcg )
with open('f_X_test_ecg.npy', 'wb') as f:
    np.save(f, f_X_test_ecg )
with open('f_X_train_ecg.npy', 'wb') as f:
    np.save(f, f_X_train_ecg )
with open('f_X_train_pcg.npy', 'wb') as f:
    np.save(f, f_X_train_pcg )
with open('f_mfcc_train.npy', 'wb') as f:
    np.save(f, f_mfcc_train )
with open('f_mfcc_test.npy', 'wb') as f:
    np.save(f, f_mfcc_test )
with open('f_y_test.npy', 'wb') as f:
    np.save(f, f_y_test )
with open('f_y_train.npy', 'wb') as f:
    np.save(f, f_y_train )

In [39]:
with open('record_id.npy', 'wb') as f:
    np.save(f, record_id)

In [40]:
with open('record_id_test.npy', 'wb') as f:
    np.save(f, record_id_test)

In [41]:
len (f_X_train_pcg) 

1793

In [133]:
record_id_test

[0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 16,
 16,
 16,
 16,
 16,
 16,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 20,
 20,
 20,
 20,
 20,
 20,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 24,
 24,
 24,
 24,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 27,
 27,
 27,
 27,
 27,
 27,
 28,
 28,
 28,
 28,
 28,
 28,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 31,
 31,
 31,
 31,
 32,
 32,
 32,
 32,
 32,

In [43]:


len(record_id)

1793

### Scalogram extraction

In [49]:
f_X_test_pcg[0]

array([[-0.1029403 ],
       [-0.02992199],
       [ 0.03697349],
       ...,
       [ 0.15162968],
       [ 0.14455582],
       [ 0.10920261]])

In [44]:

if __name__ == '__main__':
    fs = 2000
    wav = pywt.ContinuousWavelet("cmor1.5-1.0")
    scales = np.arange(20, 500,2)
    directory = 'ecg\\test'
    record_ecg_test_list = []

    with concurrent.futures.ProcessPoolExecutor(12) as executor:
            futures = [executor.submit(gen_wav, sig, r ,y, fs, wav, scales, directory, i, vmax_q=95) for sig,r, y,i in  zip(f_X_test_ecg, record_id_test, f_y_test, range(len(f_X_test_ecg)))]
            for future in concurrent.futures.as_completed(futures):
                print (future.result())
                record_ecg_test_list.append(future.result())
            

    df = pd.DataFrame(record_ecg_test_list)
    df.to_csv(directory+'\\dataset.csv')   

{'filename': '0_4.tiff', 'label': 1}
{'filename': '0_2.tiff', 'label': 1}
{'filename': '1_9.tiff', 'label': 1}
{'filename': '1_6.tiff', 'label': 1}
{'filename': '1_7.tiff', 'label': 1}
{'filename': '0_3.tiff', 'label': 1}
{'filename': '1_11.tiff', 'label': 1}
{'filename': '1_5.tiff', 'label': 1}
{'filename': '1_8.tiff', 'label': 1}
{'filename': '0_1.tiff', 'label': 1}
{'filename': '0_0.tiff', 'label': 1}
{'filename': '1_10.tiff', 'label': 1}
{'filename': '2_14.tiff', 'label': 1}
{'filename': '2_16.tiff', 'label': 1}
{'filename': '2_13.tiff', 'label': 1}
{'filename': '2_12.tiff', 'label': 1}
{'filename': '2_17.tiff', 'label': 1}
{'filename': '3_21.tiff', 'label': 1}
{'filename': '2_18.tiff', 'label': 1}
{'filename': '3_19.tiff', 'label': 1}
{'filename': '3_22.tiff', 'label': 1}
{'filename': '3_20.tiff', 'label': 1}
{'filename': '3_23.tiff', 'label': 1}
{'filename': '2_15.tiff', 'label': 1}
{'filename': '3_24.tiff', 'label': 1}
{'filename': '5_30.tiff', 'label': 0}
{'filename': '3_25.tif

{'filename': '32_211.tiff', 'label': 0}
{'filename': '32_210.tiff', 'label': 0}
{'filename': '32_212.tiff', 'label': 0}
{'filename': '33_215.tiff', 'label': 1}
{'filename': '32_214.tiff', 'label': 0}
{'filename': '32_213.tiff', 'label': 0}
{'filename': '33_216.tiff', 'label': 1}
{'filename': '33_218.tiff', 'label': 1}
{'filename': '33_217.tiff', 'label': 1}
{'filename': '33_219.tiff', 'label': 1}
{'filename': '33_220.tiff', 'label': 1}
{'filename': '34_221.tiff', 'label': 0}
{'filename': '34_222.tiff', 'label': 0}
{'filename': '34_223.tiff', 'label': 0}
{'filename': '34_225.tiff', 'label': 0}
{'filename': '34_224.tiff', 'label': 0}
{'filename': '35_227.tiff', 'label': 0}
{'filename': '35_226.tiff', 'label': 0}
{'filename': '35_228.tiff', 'label': 0}
{'filename': '35_230.tiff', 'label': 0}
{'filename': '35_229.tiff', 'label': 0}
{'filename': '35_231.tiff', 'label': 0}
{'filename': '36_232.tiff', 'label': 0}
{'filename': '36_233.tiff', 'label': 0}
{'filename': '36_234.tiff', 'label': 0}


{'filename': '67_415.tiff', 'label': 1}
{'filename': '67_416.tiff', 'label': 1}
{'filename': '67_417.tiff', 'label': 1}
{'filename': '67_418.tiff', 'label': 1}
{'filename': '67_419.tiff', 'label': 1}
{'filename': '67_420.tiff', 'label': 1}
{'filename': '68_421.tiff', 'label': 0}
{'filename': '68_422.tiff', 'label': 0}
{'filename': '68_423.tiff', 'label': 0}
{'filename': '68_425.tiff', 'label': 0}
{'filename': '68_424.tiff', 'label': 0}
{'filename': '68_426.tiff', 'label': 0}
{'filename': '68_427.tiff', 'label': 0}
{'filename': '69_428.tiff', 'label': 1}
{'filename': '69_429.tiff', 'label': 1}
{'filename': '69_430.tiff', 'label': 1}
{'filename': '69_431.tiff', 'label': 1}
{'filename': '69_432.tiff', 'label': 1}
{'filename': '69_433.tiff', 'label': 1}
{'filename': '69_434.tiff', 'label': 1}
{'filename': '70_435.tiff', 'label': 1}
{'filename': '70_437.tiff', 'label': 1}
{'filename': '70_436.tiff', 'label': 1}
{'filename': '70_439.tiff', 'label': 1}
{'filename': '70_438.tiff', 'label': 1}


{'filename': '98_620.tiff', 'label': 1}
{'filename': '98_621.tiff', 'label': 1}
{'filename': '98_622.tiff', 'label': 1}
{'filename': '99_623.tiff', 'label': 1}
{'filename': '99_624.tiff', 'label': 1}
{'filename': '99_625.tiff', 'label': 1}
{'filename': '99_626.tiff', 'label': 1}
{'filename': '99_627.tiff', 'label': 1}
{'filename': '99_628.tiff', 'label': 1}
{'filename': '99_629.tiff', 'label': 1}
{'filename': '100_630.tiff', 'label': 1}
{'filename': '100_631.tiff', 'label': 1}
{'filename': '100_632.tiff', 'label': 1}
{'filename': '100_633.tiff', 'label': 1}
{'filename': '100_634.tiff', 'label': 1}
{'filename': '100_635.tiff', 'label': 1}
{'filename': '101_638.tiff', 'label': 1}
{'filename': '100_636.tiff', 'label': 1}
{'filename': '101_637.tiff', 'label': 1}
{'filename': '101_639.tiff', 'label': 1}
{'filename': '101_641.tiff', 'label': 1}
{'filename': '101_640.tiff', 'label': 1}
{'filename': '101_642.tiff', 'label': 1}
{'filename': '101_643.tiff', 'label': 1}
{'filename': '102_644.tiff

In [45]:

if __name__ == '__main__':
    fs = 2000
    wav = pywt.ContinuousWavelet("cmor1.5-1.0")
    scales = np.arange(20, 500,2)
    directory = 'ecg\\train'
    record_ecg_test_list = []

    with concurrent.futures.ProcessPoolExecutor(12) as executor:
            futures = [executor.submit(gen_wav, sig, r ,y, fs, wav, scales, directory, i, vmax_q=95) for sig,r, y,i in  zip(f_X_train_ecg, record_id, f_y_train, range(len(f_X_train_ecg)))]
            for future in concurrent.futures.as_completed(futures):
                print (future.result())
                record_ecg_test_list.append(future.result())
            

    df = pd.DataFrame(record_ecg_test_list)
    df.to_csv(directory+'\\dataset.csv')   

{'filename': '1_11.tiff', 'label': 1}
{'filename': '1_10.tiff', 'label': 1}
{'filename': '1_9.tiff', 'label': 1}
{'filename': '0_4.tiff', 'label': 1}
{'filename': '0_2.tiff', 'label': 1}
{'filename': '0_0.tiff', 'label': 1}
{'filename': '0_1.tiff', 'label': 1}
{'filename': '1_8.tiff', 'label': 1}
{'filename': '1_7.tiff', 'label': 1}
{'filename': '0_5.tiff', 'label': 1}
{'filename': '0_6.tiff', 'label': 1}
{'filename': '0_3.tiff', 'label': 1}
{'filename': '2_14.tiff', 'label': 1}
{'filename': '2_15.tiff', 'label': 1}
{'filename': '2_16.tiff', 'label': 1}
{'filename': '1_12.tiff', 'label': 1}
{'filename': '4_22.tiff', 'label': 0}
{'filename': '1_13.tiff', 'label': 1}
{'filename': '3_18.tiff', 'label': 1}
{'filename': '3_20.tiff', 'label': 1}
{'filename': '3_19.tiff', 'label': 1}
{'filename': '3_21.tiff', 'label': 1}
{'filename': '2_17.tiff', 'label': 1}
{'filename': '4_23.tiff', 'label': 0}
{'filename': '4_24.tiff', 'label': 0}
{'filename': '4_25.tiff', 'label': 0}
{'filename': '4_26.tif

{'filename': '32_209.tiff', 'label': 1}
{'filename': '32_211.tiff', 'label': 1}
{'filename': '32_212.tiff', 'label': 1}
{'filename': '32_213.tiff', 'label': 1}
{'filename': '33_214.tiff', 'label': 0}
{'filename': '33_215.tiff', 'label': 0}
{'filename': '33_216.tiff', 'label': 0}
{'filename': '33_217.tiff', 'label': 0}
{'filename': '33_218.tiff', 'label': 0}
{'filename': '33_219.tiff', 'label': 0}
{'filename': '34_220.tiff', 'label': 1}
{'filename': '34_221.tiff', 'label': 1}
{'filename': '34_222.tiff', 'label': 1}
{'filename': '34_223.tiff', 'label': 1}
{'filename': '35_224.tiff', 'label': 0}
{'filename': '35_226.tiff', 'label': 0}
{'filename': '35_225.tiff', 'label': 0}
{'filename': '35_227.tiff', 'label': 0}
{'filename': '35_228.tiff', 'label': 0}
{'filename': '35_229.tiff', 'label': 0}
{'filename': '36_230.tiff', 'label': 1}
{'filename': '36_231.tiff', 'label': 1}
{'filename': '36_232.tiff', 'label': 1}
{'filename': '36_233.tiff', 'label': 1}
{'filename': '36_234.tiff', 'label': 1}


{'filename': '65_415.tiff', 'label': 1}
{'filename': '65_416.tiff', 'label': 1}
{'filename': '65_417.tiff', 'label': 1}
{'filename': '65_418.tiff', 'label': 1}
{'filename': '65_419.tiff', 'label': 1}
{'filename': '65_420.tiff', 'label': 1}
{'filename': '65_421.tiff', 'label': 1}
{'filename': '66_422.tiff', 'label': 1}
{'filename': '66_424.tiff', 'label': 1}
{'filename': '66_423.tiff', 'label': 1}
{'filename': '66_425.tiff', 'label': 1}
{'filename': '66_426.tiff', 'label': 1}
{'filename': '66_427.tiff', 'label': 1}
{'filename': '67_428.tiff', 'label': 1}
{'filename': '67_429.tiff', 'label': 1}
{'filename': '67_430.tiff', 'label': 1}
{'filename': '67_431.tiff', 'label': 1}
{'filename': '67_432.tiff', 'label': 1}
{'filename': '67_433.tiff', 'label': 1}
{'filename': '67_434.tiff', 'label': 1}
{'filename': '68_435.tiff', 'label': 1}
{'filename': '68_436.tiff', 'label': 1}
{'filename': '68_437.tiff', 'label': 1}
{'filename': '68_438.tiff', 'label': 1}
{'filename': '69_439.tiff', 'label': 0}


{'filename': '98_621.tiff', 'label': 1}
{'filename': '99_622.tiff', 'label': 0}
{'filename': '99_623.tiff', 'label': 0}
{'filename': '99_624.tiff', 'label': 0}
{'filename': '99_625.tiff', 'label': 0}
{'filename': '99_626.tiff', 'label': 0}
{'filename': '99_628.tiff', 'label': 0}
{'filename': '99_627.tiff', 'label': 0}
{'filename': '100_629.tiff', 'label': 1}
{'filename': '100_630.tiff', 'label': 1}
{'filename': '100_631.tiff', 'label': 1}
{'filename': '100_632.tiff', 'label': 1}
{'filename': '100_633.tiff', 'label': 1}
{'filename': '100_634.tiff', 'label': 1}
{'filename': '100_635.tiff', 'label': 1}
{'filename': '101_636.tiff', 'label': 1}
{'filename': '101_637.tiff', 'label': 1}
{'filename': '101_638.tiff', 'label': 1}
{'filename': '101_639.tiff', 'label': 1}
{'filename': '101_640.tiff', 'label': 1}
{'filename': '101_642.tiff', 'label': 1}
{'filename': '101_641.tiff', 'label': 1}
{'filename': '102_643.tiff', 'label': 1}
{'filename': '102_644.tiff', 'label': 1}
{'filename': '102_645.ti

{'filename': '131_825.tiff', 'label': 0}
{'filename': '130_824.tiff', 'label': 1}
{'filename': '131_826.tiff', 'label': 0}
{'filename': '131_827.tiff', 'label': 0}
{'filename': '131_828.tiff', 'label': 0}
{'filename': '132_829.tiff', 'label': 1}
{'filename': '132_830.tiff', 'label': 1}
{'filename': '132_831.tiff', 'label': 1}
{'filename': '132_832.tiff', 'label': 1}
{'filename': '132_834.tiff', 'label': 1}
{'filename': '132_835.tiff', 'label': 1}
{'filename': '132_833.tiff', 'label': 1}
{'filename': '133_836.tiff', 'label': 1}
{'filename': '133_837.tiff', 'label': 1}
{'filename': '133_838.tiff', 'label': 1}
{'filename': '133_839.tiff', 'label': 1}
{'filename': '133_840.tiff', 'label': 1}
{'filename': '133_841.tiff', 'label': 1}
{'filename': '133_842.tiff', 'label': 1}
{'filename': '134_843.tiff', 'label': 1}
{'filename': '134_844.tiff', 'label': 1}
{'filename': '134_845.tiff', 'label': 1}
{'filename': '134_846.tiff', 'label': 1}
{'filename': '134_847.tiff', 'label': 1}
{'filename': '13

{'filename': '162_1024.tiff', 'label': 1}
{'filename': '162_1025.tiff', 'label': 1}
{'filename': '162_1026.tiff', 'label': 1}
{'filename': '162_1027.tiff', 'label': 1}
{'filename': '163_1028.tiff', 'label': 1}
{'filename': '163_1030.tiff', 'label': 1}
{'filename': '163_1029.tiff', 'label': 1}
{'filename': '163_1031.tiff', 'label': 1}
{'filename': '163_1032.tiff', 'label': 1}
{'filename': '163_1033.tiff', 'label': 1}
{'filename': '164_1035.tiff', 'label': 1}
{'filename': '163_1034.tiff', 'label': 1}
{'filename': '164_1036.tiff', 'label': 1}
{'filename': '164_1037.tiff', 'label': 1}
{'filename': '165_1040.tiff', 'label': 0}
{'filename': '164_1039.tiff', 'label': 1}
{'filename': '164_1038.tiff', 'label': 1}
{'filename': '165_1041.tiff', 'label': 0}
{'filename': '165_1042.tiff', 'label': 0}
{'filename': '165_1043.tiff', 'label': 0}
{'filename': '165_1044.tiff', 'label': 0}
{'filename': '165_1045.tiff', 'label': 0}
{'filename': '166_1047.tiff', 'label': 0}
{'filename': '165_1046.tiff', 'lab

{'filename': '191_1220.tiff', 'label': 0}
{'filename': '192_1222.tiff', 'label': 1}
{'filename': '191_1221.tiff', 'label': 0}
{'filename': '192_1223.tiff', 'label': 1}
{'filename': '192_1224.tiff', 'label': 1}
{'filename': '192_1225.tiff', 'label': 1}
{'filename': '193_1226.tiff', 'label': 1}
{'filename': '193_1227.tiff', 'label': 1}
{'filename': '193_1228.tiff', 'label': 1}
{'filename': '193_1229.tiff', 'label': 1}
{'filename': '193_1231.tiff', 'label': 1}
{'filename': '193_1230.tiff', 'label': 1}
{'filename': '194_1232.tiff', 'label': 0}
{'filename': '194_1233.tiff', 'label': 0}
{'filename': '194_1234.tiff', 'label': 0}
{'filename': '194_1235.tiff', 'label': 0}
{'filename': '194_1236.tiff', 'label': 0}
{'filename': '194_1237.tiff', 'label': 0}
{'filename': '195_1238.tiff', 'label': 1}
{'filename': '195_1239.tiff', 'label': 1}
{'filename': '195_1240.tiff', 'label': 1}
{'filename': '195_1241.tiff', 'label': 1}
{'filename': '195_1242.tiff', 'label': 1}
{'filename': '195_1243.tiff', 'lab

{'filename': '225_1417.tiff', 'label': 1}
{'filename': '225_1416.tiff', 'label': 1}
{'filename': '225_1418.tiff', 'label': 1}
{'filename': '225_1419.tiff', 'label': 1}
{'filename': '226_1420.tiff', 'label': 1}
{'filename': '226_1421.tiff', 'label': 1}
{'filename': '226_1422.tiff', 'label': 1}
{'filename': '226_1423.tiff', 'label': 1}
{'filename': '226_1424.tiff', 'label': 1}
{'filename': '226_1426.tiff', 'label': 1}
{'filename': '226_1425.tiff', 'label': 1}
{'filename': '227_1427.tiff', 'label': 1}
{'filename': '227_1429.tiff', 'label': 1}
{'filename': '227_1428.tiff', 'label': 1}
{'filename': '227_1430.tiff', 'label': 1}
{'filename': '228_1431.tiff', 'label': 1}
{'filename': '228_1432.tiff', 'label': 1}
{'filename': '228_1433.tiff', 'label': 1}
{'filename': '228_1436.tiff', 'label': 1}
{'filename': '228_1435.tiff', 'label': 1}
{'filename': '228_1434.tiff', 'label': 1}
{'filename': '228_1437.tiff', 'label': 1}
{'filename': '229_1438.tiff', 'label': 1}
{'filename': '229_1439.tiff', 'lab

{'filename': '255_1612.tiff', 'label': 1}
{'filename': '255_1613.tiff', 'label': 1}
{'filename': '255_1614.tiff', 'label': 1}
{'filename': '255_1615.tiff', 'label': 1}
{'filename': '255_1616.tiff', 'label': 1}
{'filename': '256_1617.tiff', 'label': 1}
{'filename': '256_1618.tiff', 'label': 1}
{'filename': '256_1621.tiff', 'label': 1}
{'filename': '256_1619.tiff', 'label': 1}
{'filename': '256_1623.tiff', 'label': 1}
{'filename': '256_1620.tiff', 'label': 1}
{'filename': '256_1622.tiff', 'label': 1}
{'filename': '257_1624.tiff', 'label': 0}
{'filename': '257_1625.tiff', 'label': 0}
{'filename': '257_1626.tiff', 'label': 0}
{'filename': '257_1627.tiff', 'label': 0}
{'filename': '257_1628.tiff', 'label': 0}
{'filename': '257_1629.tiff', 'label': 0}
{'filename': '258_1632.tiff', 'label': 1}
{'filename': '258_1633.tiff', 'label': 1}
{'filename': '258_1635.tiff', 'label': 1}
{'filename': '258_1630.tiff', 'label': 1}
{'filename': '258_1631.tiff', 'label': 1}
{'filename': '258_1636.tiff', 'lab

In [46]:

if __name__ == '__main__':
    fs = 2000
    wav = pywt.ContinuousWavelet("morl")
    scales = np.arange(7, 100)
    directory = 'pcg\\test'
    record_ecg_test_list = []

    with concurrent.futures.ProcessPoolExecutor(12) as executor:
            futures = [executor.submit(gen_wav, sig, r ,y, fs, wav, scales, directory, i) for sig,r, y,i in  zip(f_X_test_pcg, record_id_test, f_y_test, range(len(f_X_test_pcg)))]
            for future in concurrent.futures.as_completed(futures):
                print (future.result())
                record_ecg_test_list.append(future.result())
            

    df = pd.DataFrame(record_ecg_test_list)
    df.to_csv(directory+'\\dataset.csv')   

{'filename': '0_1.tiff', 'label': 1}
{'filename': '0_3.tiff', 'label': 1}
{'filename': '0_2.tiff', 'label': 1}
{'filename': '0_0.tiff', 'label': 1}
{'filename': '0_4.tiff', 'label': 1}
{'filename': '1_7.tiff', 'label': 1}
{'filename': '1_9.tiff', 'label': 1}
{'filename': '1_5.tiff', 'label': 1}
{'filename': '1_8.tiff', 'label': 1}
{'filename': '1_6.tiff', 'label': 1}
{'filename': '1_10.tiff', 'label': 1}
{'filename': '1_11.tiff', 'label': 1}
{'filename': '2_12.tiff', 'label': 1}
{'filename': '2_13.tiff', 'label': 1}
{'filename': '2_14.tiff', 'label': 1}
{'filename': '2_15.tiff', 'label': 1}
{'filename': '2_16.tiff', 'label': 1}
{'filename': '2_17.tiff', 'label': 1}
{'filename': '2_18.tiff', 'label': 1}
{'filename': '3_19.tiff', 'label': 1}
{'filename': '3_20.tiff', 'label': 1}
{'filename': '3_22.tiff', 'label': 1}
{'filename': '3_21.tiff', 'label': 1}
{'filename': '3_23.tiff', 'label': 1}
{'filename': '3_24.tiff', 'label': 1}
{'filename': '3_25.tiff', 'label': 1}
{'filename': '4_26.tif

{'filename': '32_213.tiff', 'label': 0}
{'filename': '32_214.tiff', 'label': 0}
{'filename': '33_216.tiff', 'label': 1}
{'filename': '33_215.tiff', 'label': 1}
{'filename': '33_218.tiff', 'label': 1}
{'filename': '33_217.tiff', 'label': 1}
{'filename': '33_219.tiff', 'label': 1}
{'filename': '33_220.tiff', 'label': 1}
{'filename': '34_221.tiff', 'label': 0}
{'filename': '34_222.tiff', 'label': 0}
{'filename': '34_223.tiff', 'label': 0}
{'filename': '34_224.tiff', 'label': 0}
{'filename': '34_225.tiff', 'label': 0}
{'filename': '35_226.tiff', 'label': 0}
{'filename': '35_227.tiff', 'label': 0}
{'filename': '35_230.tiff', 'label': 0}
{'filename': '35_229.tiff', 'label': 0}
{'filename': '35_228.tiff', 'label': 0}
{'filename': '35_231.tiff', 'label': 0}
{'filename': '36_232.tiff', 'label': 0}
{'filename': '36_233.tiff', 'label': 0}
{'filename': '36_234.tiff', 'label': 0}
{'filename': '36_236.tiff', 'label': 0}
{'filename': '36_235.tiff', 'label': 0}
{'filename': '36_237.tiff', 'label': 0}


{'filename': '67_418.tiff', 'label': 1}
{'filename': '67_419.tiff', 'label': 1}
{'filename': '68_421.tiff', 'label': 0}
{'filename': '67_420.tiff', 'label': 1}
{'filename': '68_422.tiff', 'label': 0}
{'filename': '68_423.tiff', 'label': 0}
{'filename': '68_424.tiff', 'label': 0}
{'filename': '68_425.tiff', 'label': 0}
{'filename': '68_426.tiff', 'label': 0}
{'filename': '69_428.tiff', 'label': 1}
{'filename': '68_427.tiff', 'label': 0}
{'filename': '69_429.tiff', 'label': 1}
{'filename': '69_430.tiff', 'label': 1}
{'filename': '69_433.tiff', 'label': 1}
{'filename': '69_431.tiff', 'label': 1}
{'filename': '69_432.tiff', 'label': 1}
{'filename': '70_437.tiff', 'label': 1}
{'filename': '70_436.tiff', 'label': 1}
{'filename': '70_435.tiff', 'label': 1}
{'filename': '70_439.tiff', 'label': 1}
{'filename': '69_434.tiff', 'label': 1}
{'filename': '70_440.tiff', 'label': 1}
{'filename': '70_438.tiff', 'label': 1}
{'filename': '70_441.tiff', 'label': 1}
{'filename': '71_442.tiff', 'label': 1}


{'filename': '100_630.tiff', 'label': 1}
{'filename': '100_631.tiff', 'label': 1}
{'filename': '100_632.tiff', 'label': 1}
{'filename': '100_634.tiff', 'label': 1}
{'filename': '100_633.tiff', 'label': 1}
{'filename': '100_635.tiff', 'label': 1}
{'filename': '101_637.tiff', 'label': 1}
{'filename': '100_636.tiff', 'label': 1}
{'filename': '101_638.tiff', 'label': 1}
{'filename': '101_639.tiff', 'label': 1}
{'filename': '101_640.tiff', 'label': 1}
{'filename': '101_641.tiff', 'label': 1}
{'filename': '101_642.tiff', 'label': 1}
{'filename': '101_643.tiff', 'label': 1}
{'filename': '102_644.tiff', 'label': 1}
{'filename': '102_645.tiff', 'label': 1}
{'filename': '102_648.tiff', 'label': 1}
{'filename': '102_646.tiff', 'label': 1}
{'filename': '102_647.tiff', 'label': 1}
{'filename': '102_650.tiff', 'label': 1}
{'filename': '102_649.tiff', 'label': 1}
{'filename': '103_651.tiff', 'label': 0}
{'filename': '103_653.tiff', 'label': 0}
{'filename': '103_652.tiff', 'label': 0}
{'filename': '10

In [47]:

if __name__ == '__main__':
    fs = 2000
    wav = pywt.ContinuousWavelet("morl")
    scales = np.arange(7, 100)
    directory = 'pcg\\train'
    record_ecg_test_list = []

    with concurrent.futures.ProcessPoolExecutor(12) as executor:
            futures = [executor.submit(gen_wav, sig, r ,y, fs, wav, scales, directory, i) for sig,r, y,i in  zip(f_X_train_pcg, record_id, f_y_train, range(len(f_X_train_pcg)))]
            for future in concurrent.futures.as_completed(futures):
                print (future.result())
                record_ecg_test_list.append(future.result())
            

    df = pd.DataFrame(record_ecg_test_list)
    df.to_csv(directory+'\\dataset.csv')   


{'filename': '0_2.tiff', 'label': 1}
{'filename': '0_0.tiff', 'label': 1}
{'filename': '1_11.tiff', 'label': 1}
{'filename': '1_7.tiff', 'label': 1}
{'filename': '1_9.tiff', 'label': 1}
{'filename': '0_6.tiff', 'label': 1}
{'filename': '1_10.tiff', 'label': 1}
{'filename': '1_8.tiff', 'label': 1}
{'filename': '0_1.tiff', 'label': 1}
{'filename': '0_5.tiff', 'label': 1}
{'filename': '0_4.tiff', 'label': 1}
{'filename': '0_3.tiff', 'label': 1}
{'filename': '1_12.tiff', 'label': 1}
{'filename': '1_13.tiff', 'label': 1}
{'filename': '2_14.tiff', 'label': 1}
{'filename': '2_15.tiff', 'label': 1}
{'filename': '2_16.tiff', 'label': 1}
{'filename': '2_17.tiff', 'label': 1}
{'filename': '3_19.tiff', 'label': 1}
{'filename': '3_18.tiff', 'label': 1}
{'filename': '3_21.tiff', 'label': 1}
{'filename': '3_20.tiff', 'label': 1}
{'filename': '4_22.tiff', 'label': 0}
{'filename': '4_23.tiff', 'label': 0}
{'filename': '4_24.tiff', 'label': 0}
{'filename': '4_25.tiff', 'label': 0}
{'filename': '4_26.tif

{'filename': '32_213.tiff', 'label': 1}
{'filename': '33_214.tiff', 'label': 0}
{'filename': '33_216.tiff', 'label': 0}
{'filename': '33_217.tiff', 'label': 0}
{'filename': '33_215.tiff', 'label': 0}
{'filename': '34_220.tiff', 'label': 1}
{'filename': '34_222.tiff', 'label': 1}
{'filename': '33_219.tiff', 'label': 0}
{'filename': '35_224.tiff', 'label': 0}
{'filename': '34_223.tiff', 'label': 1}
{'filename': '33_218.tiff', 'label': 0}
{'filename': '34_221.tiff', 'label': 1}
{'filename': '35_225.tiff', 'label': 0}
{'filename': '35_226.tiff', 'label': 0}
{'filename': '35_228.tiff', 'label': 0}
{'filename': '35_227.tiff', 'label': 0}
{'filename': '35_229.tiff', 'label': 0}
{'filename': '36_230.tiff', 'label': 1}
{'filename': '36_231.tiff', 'label': 1}
{'filename': '36_232.tiff', 'label': 1}
{'filename': '36_235.tiff', 'label': 1}
{'filename': '36_233.tiff', 'label': 1}
{'filename': '36_234.tiff', 'label': 1}
{'filename': '37_236.tiff', 'label': 0}
{'filename': '37_237.tiff', 'label': 0}


{'filename': '67_431.tiff', 'label': 1}
{'filename': '67_433.tiff', 'label': 1}
{'filename': '66_426.tiff', 'label': 1}
{'filename': '66_427.tiff', 'label': 1}
{'filename': '67_429.tiff', 'label': 1}
{'filename': '67_428.tiff', 'label': 1}
{'filename': '67_432.tiff', 'label': 1}
{'filename': '67_430.tiff', 'label': 1}
{'filename': '68_435.tiff', 'label': 1}
{'filename': '67_434.tiff', 'label': 1}
{'filename': '68_436.tiff', 'label': 1}
{'filename': '68_437.tiff', 'label': 1}
{'filename': '69_439.tiff', 'label': 0}
{'filename': '69_440.tiff', 'label': 0}
{'filename': '69_442.tiff', 'label': 0}
{'filename': '68_438.tiff', 'label': 1}
{'filename': '69_441.tiff', 'label': 0}
{'filename': '70_445.tiff', 'label': 1}
{'filename': '69_444.tiff', 'label': 0}
{'filename': '69_443.tiff', 'label': 0}
{'filename': '70_447.tiff', 'label': 1}
{'filename': '70_446.tiff', 'label': 1}
{'filename': '70_448.tiff', 'label': 1}
{'filename': '71_449.tiff', 'label': 0}
{'filename': '71_450.tiff', 'label': 0}


{'filename': '100_631.tiff', 'label': 1}
{'filename': '100_634.tiff', 'label': 1}
{'filename': '100_633.tiff', 'label': 1}
{'filename': '101_636.tiff', 'label': 1}
{'filename': '100_635.tiff', 'label': 1}
{'filename': '101_637.tiff', 'label': 1}
{'filename': '101_638.tiff', 'label': 1}
{'filename': '101_639.tiff', 'label': 1}
{'filename': '101_640.tiff', 'label': 1}
{'filename': '101_641.tiff', 'label': 1}
{'filename': '101_642.tiff', 'label': 1}
{'filename': '102_643.tiff', 'label': 1}
{'filename': '102_644.tiff', 'label': 1}
{'filename': '102_646.tiff', 'label': 1}
{'filename': '102_645.tiff', 'label': 1}
{'filename': '103_649.tiff', 'label': 1}
{'filename': '102_647.tiff', 'label': 1}
{'filename': '102_648.tiff', 'label': 1}
{'filename': '103_650.tiff', 'label': 1}
{'filename': '103_651.tiff', 'label': 1}
{'filename': '103_652.tiff', 'label': 1}
{'filename': '103_653.tiff', 'label': 1}
{'filename': '104_657.tiff', 'label': 1}
{'filename': '103_654.tiff', 'label': 1}
{'filename': '10

{'filename': '133_838.tiff', 'label': 1}
{'filename': '132_835.tiff', 'label': 1}
{'filename': '133_836.tiff', 'label': 1}
{'filename': '133_837.tiff', 'label': 1}
{'filename': '132_834.tiff', 'label': 1}
{'filename': '133_839.tiff', 'label': 1}
{'filename': '133_840.tiff', 'label': 1}
{'filename': '133_842.tiff', 'label': 1}
{'filename': '133_841.tiff', 'label': 1}
{'filename': '134_843.tiff', 'label': 1}
{'filename': '134_844.tiff', 'label': 1}
{'filename': '134_845.tiff', 'label': 1}
{'filename': '134_847.tiff', 'label': 1}
{'filename': '134_848.tiff', 'label': 1}
{'filename': '134_846.tiff', 'label': 1}
{'filename': '135_850.tiff', 'label': 0}
{'filename': '134_849.tiff', 'label': 1}
{'filename': '135_855.tiff', 'label': 0}
{'filename': '135_854.tiff', 'label': 0}
{'filename': '135_851.tiff', 'label': 0}
{'filename': '135_852.tiff', 'label': 0}
{'filename': '135_853.tiff', 'label': 0}
{'filename': '136_857.tiff', 'label': 1}
{'filename': '136_856.tiff', 'label': 1}
{'filename': '13

{'filename': '164_1036.tiff', 'label': 1}
{'filename': '164_1037.tiff', 'label': 1}
{'filename': '164_1039.tiff', 'label': 1}
{'filename': '165_1040.tiff', 'label': 0}
{'filename': '164_1038.tiff', 'label': 1}
{'filename': '165_1041.tiff', 'label': 0}
{'filename': '165_1043.tiff', 'label': 0}
{'filename': '165_1042.tiff', 'label': 0}
{'filename': '165_1045.tiff', 'label': 0}
{'filename': '165_1044.tiff', 'label': 0}
{'filename': '165_1046.tiff', 'label': 0}
{'filename': '166_1047.tiff', 'label': 0}
{'filename': '166_1051.tiff', 'label': 0}
{'filename': '166_1048.tiff', 'label': 0}
{'filename': '166_1049.tiff', 'label': 0}
{'filename': '166_1052.tiff', 'label': 0}
{'filename': '166_1050.tiff', 'label': 0}
{'filename': '167_1054.tiff', 'label': 1}
{'filename': '166_1053.tiff', 'label': 0}
{'filename': '167_1055.tiff', 'label': 1}
{'filename': '167_1057.tiff', 'label': 1}
{'filename': '167_1056.tiff', 'label': 1}
{'filename': '167_1058.tiff', 'label': 1}
{'filename': '167_1059.tiff', 'lab

{'filename': '195_1238.tiff', 'label': 1}
{'filename': '194_1236.tiff', 'label': 0}
{'filename': '194_1235.tiff', 'label': 0}
{'filename': '194_1237.tiff', 'label': 0}
{'filename': '195_1239.tiff', 'label': 1}
{'filename': '195_1240.tiff', 'label': 1}
{'filename': '195_1241.tiff', 'label': 1}
{'filename': '195_1242.tiff', 'label': 1}
{'filename': '195_1244.tiff', 'label': 1}
{'filename': '195_1243.tiff', 'label': 1}
{'filename': '196_1245.tiff', 'label': 0}
{'filename': '196_1246.tiff', 'label': 0}
{'filename': '196_1247.tiff', 'label': 0}
{'filename': '197_1252.tiff', 'label': 1}
{'filename': '196_1248.tiff', 'label': 0}
{'filename': '196_1249.tiff', 'label': 0}
{'filename': '197_1253.tiff', 'label': 1}
{'filename': '196_1250.tiff', 'label': 0}
{'filename': '196_1251.tiff', 'label': 0}
{'filename': '197_1254.tiff', 'label': 1}
{'filename': '197_1255.tiff', 'label': 1}
{'filename': '198_1257.tiff', 'label': 1}
{'filename': '198_1256.tiff', 'label': 1}
{'filename': '198_1258.tiff', 'lab

{'filename': '228_1437.tiff', 'label': 1}
{'filename': '229_1438.tiff', 'label': 1}
{'filename': '229_1439.tiff', 'label': 1}
{'filename': '229_1440.tiff', 'label': 1}
{'filename': '229_1441.tiff', 'label': 1}
{'filename': '229_1442.tiff', 'label': 1}
{'filename': '229_1443.tiff', 'label': 1}
{'filename': '229_1444.tiff', 'label': 1}
{'filename': '230_1446.tiff', 'label': 0}
{'filename': '230_1445.tiff', 'label': 0}
{'filename': '230_1447.tiff', 'label': 0}
{'filename': '230_1448.tiff', 'label': 0}
{'filename': '230_1449.tiff', 'label': 0}
{'filename': '230_1450.tiff', 'label': 0}
{'filename': '230_1451.tiff', 'label': 0}
{'filename': '231_1452.tiff', 'label': 1}
{'filename': '231_1453.tiff', 'label': 1}
{'filename': '231_1455.tiff', 'label': 1}
{'filename': '231_1456.tiff', 'label': 1}
{'filename': '231_1458.tiff', 'label': 1}
{'filename': '231_1457.tiff', 'label': 1}
{'filename': '231_1454.tiff', 'label': 1}
{'filename': '232_1459.tiff', 'label': 1}
{'filename': '232_1460.tiff', 'lab

{'filename': '259_1638.tiff', 'label': 0}
{'filename': '259_1639.tiff', 'label': 0}
{'filename': '259_1642.tiff', 'label': 0}
{'filename': '259_1640.tiff', 'label': 0}
{'filename': '260_1646.tiff', 'label': 1}
{'filename': '259_1641.tiff', 'label': 0}
{'filename': '260_1648.tiff', 'label': 1}
{'filename': '260_1645.tiff', 'label': 1}
{'filename': '259_1643.tiff', 'label': 0}
{'filename': '260_1644.tiff', 'label': 1}
{'filename': '260_1647.tiff', 'label': 1}
{'filename': '260_1649.tiff', 'label': 1}
{'filename': '260_1650.tiff', 'label': 1}
{'filename': '261_1651.tiff', 'label': 0}
{'filename': '261_1654.tiff', 'label': 0}
{'filename': '261_1653.tiff', 'label': 0}
{'filename': '261_1652.tiff', 'label': 0}
{'filename': '261_1655.tiff', 'label': 0}
{'filename': '261_1656.tiff', 'label': 0}
{'filename': '262_1658.tiff', 'label': 0}
{'filename': '261_1657.tiff', 'label': 0}
{'filename': '262_1661.tiff', 'label': 0}
{'filename': '262_1659.tiff', 'label': 0}
{'filename': '262_1660.tiff', 'lab